In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

pd.set_option("display.width", 120)
pd.set_option("display.max_columns", 50)

In [3]:
ingredients = pd.DataFrame({
    "ingredient": ["pea_protein", "cocoa", "salt", "oil", "flavor", "starch"],
    "cost per kg": [7.5, 5.0, 0.4, 2.2, 18.0, 1.8],
    "sodium_mg_per_100g": [50, 20, 38750, 2, 10, 5],
    "protein_g_per_100g": [80, 20, 0, 0, 0, 1],
})
ingredients

,ingredient,cost per kg,sodium_mg_per_100g,protein_g_per_100g
0,pea_protein,7.5,50,80
1,cocoa,5.0,20,20
2,salt,0.4,38750,0
3,oil,2.2,2,0
4,flavor,18.0,10,0
5,starch,1.8,5,1


In [4]:
recipe_lines = pd.DataFrame({
    "recipe_id": [101,101,101,101,  102,102,102,102,  103,103,103,103],
    "ingredient": ["pea_protein","cocoa","oil","salt", "pea_protein","starch","oil","flavor", "starch","cocoa","oil","salt"],
    "grams":      [35,         10,     10,   1.2,     20,          50,      8,   0.8,      60,      12,     6,   1.0],
})
recipe_lines

,recipe_id,ingredient,grams
0,101,pea_protein,35.0
1,101,cocoa,10.0
2,101,oil,10.0
3,101,salt,1.2
4,102,pea_protein,20.0
5,102,starch,50.0
6,102,oil,8.0
7,102,flavor,0.8
8,103,starch,60.0
9,103,cocoa,12.0


In [5]:
df = recipe_lines.merge(ingredients, on="ingredient", how="left")
df

,recipe_id,ingredient,grams,cost per kg,sodium_mg_per_100g,protein_g_per_100g
0,101,pea_protein,35.0,7.5,50,80
1,101,cocoa,10.0,5.0,20,20
2,101,oil,10.0,2.2,2,0
3,101,salt,1.2,0.4,38750,0
4,102,pea_protein,20.0,7.5,50,80
5,102,starch,50.0,1.8,5,1
6,102,oil,8.0,2.2,2,0
7,102,flavor,0.8,18.0,10,0
8,103,starch,60.0,1.8,5,1
9,103,cocoa,12.0,5.0,20,20


In [6]:
df["cost_eur"]= df["grams"] * df["cost per kg"] / 1000
df["sodium_mg"] = df["grams"] * df["sodium_mg_per_100g"] / 100
df["protein_g"] = df["grams"] * df["protein_g_per_100g"] / 100

df[["recipe_id", "ingredient", "grams", "cost_eur", "sodium_mg", "protein_g"    ]]

,recipe_id,ingredient,grams,cost_eur,sodium_mg,protein_g
0,101,pea_protein,35.0,0.26250,17.50,28.0
1,101,cocoa,10.0,0.05000,2.00,2.0
2,101,oil,10.0,0.02200,0.20,0.0
3,101,salt,1.2,0.00048,465.00,0.0
4,102,pea_protein,20.0,0.15000,10.00,16.0
5,102,starch,50.0,0.09000,2.50,0.5
6,102,oil,8.0,0.01760,0.16,0.0
7,102,flavor,0.8,0.01440,0.08,0.0
8,103,starch,60.0,0.10800,3.00,0.6
9,103,cocoa,12.0,0.06000,2.40,2.4


Find the cost of recipe with id 101

In [7]:
# Filter for recipe 101
recipe_101 = df[df['recipe_id'] == 101].copy()

# Calculate cost for each line: (grams / 1000 kg) * cost per kg
recipe_101['cost'] = (recipe_101['grams'] / 1000) * recipe_101['cost per kg']

# Total cost
total_cost_101 = recipe_101['cost'].sum()
print(f"Recipe 101 total cost: ${total_cost_101:.2f}")

# Or as a one-liner:
total_cost_101 = ((df[df['recipe_id'] == 101]['grams'] / 1000) * df[df['recipe_id'] == 101]['cost per kg']).sum()

Recipe 101 total cost: $0.33


In [8]:
recipe_totals= df.groupby('recipe_id', as_index=False)[['grams','cost_eur', 'sodium_mg', 'protein_g']].sum()
recipe_totals

,recipe_id,grams,cost_eur,sodium_mg,protein_g
0,101,56.2,0.33498,484.70,30.0
1,102,78.8,0.27200,12.74,16.5
2,103,79.0,0.18160,393.02,3.0


In [9]:
MAX_SODIUM_MG = 600
MIN_PROTEIN_G = 15

recipe_totals["passes_sodium"] = recipe_totals["sodium_mg"] <= MAX_SODIUM_MG
recipe_totals["passes_protein"] = recipe_totals["protein_g"] >= MIN_PROTEIN_G
recipe_totals["passes_all"] = recipe_totals["passes_sodium"] & recipe_totals["passes_protein"]

recipe_totals


,recipe_id,grams,cost_eur,sodium_mg,protein_g,passes_sodium,passes_protein,passes_all
0,101,56.2,0.33498,484.70,30.0,True,True,True
1,102,78.8,0.27200,12.74,16.5,True,True,True
2,103,79.0,0.18160,393.02,3.0,True,False,False


In [10]:
recipe_totals["protein_per_eur"] = recipe_totals["protein_g"] / recipe_totals["cost_eur"]

ranked = recipe_totals.sort_values(["passes_all", "protein_per_eur"], ascending=[False, False])
ranked


,recipe_id,grams,cost_eur,sodium_mg,protein_g,passes_sodium,passes_protein,passes_all,protein_per_eur
0,101,56.2,0.33498,484.70,30.0,True,True,True,89.557586
1,102,78.8,0.27200,12.74,16.5,True,True,True,60.661765
2,103,79.0,0.18160,393.02,3.0,True,False,False,16.519824


In [11]:
ranked.to_csv("day3_recipe_totals.csv", index=False)
print("Saved: day3_recipe_totals.csv")


Saved: day3_recipe_totals.csv


In [12]:
recipe_totals["protein_per_eur"] = recipe_totals["protein_g"] / recipe_totals["cost_eur"]
recipe_totals.sort_values(["passes_all", "protein_per_eur"], ascending=[False, False])


,recipe_id,grams,cost_eur,sodium_mg,protein_g,passes_sodium,passes_protein,passes_all,protein_per_eur
0,101,56.2,0.33498,484.70,30.0,True,True,True,89.557586
1,102,78.8,0.27200,12.74,16.5,True,True,True,60.661765
2,103,79.0,0.18160,393.02,3.0,True,False,False,16.519824


In [13]:
culprits = (
    df.sort_values(["recipe_id", "sodium_mg"], ascending=[True, False])
      .groupby("recipe_id", as_index=False)
      .head(1)[["recipe_id", "ingredient", "grams", "sodium_mg"]]
)
culprits


,recipe_id,ingredient,grams,sodium_mg
3,101,salt,1.2,465.0
4,102,pea_protein,20.0,10.0
11,103,salt,1.0,387.5
